In [1]:
from qgis.core import *
import pandas as pd
import numpy as np

In [2]:
import sys
import os
from osgeo import ogr
from qgis.core import (
     QgsApplication,
     QgsProcessingFeedback,
     QgsVectorLayer
)
from qgis.analysis import QgsNativeAlgorithms

In [3]:
QgsApplication.setPrefixPath(r'C:\OSGeo4W\apps\qgis', True)
qgs = QgsApplication([], False)
qgs.initQgis()
sys.path.append(r'C:\OSGeo4W\apps\qgis\python\plugins')

In [4]:
import geopandas as gpd
import matplotlib.pyplot as plt
plt.ion()
import xlwings as xw
import time
import seaborn as sns
import math
import statistics
import scipy.stats

import processing
from processing.core.Processing import Processing
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
#for alg in QgsApplication.processingRegistry().algorithms():
#        print(alg.id(), "--->", alg.displayName())

C:\OSGeo4W\apps\Python39\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


False

In [5]:
# living_houses_pts = QgsVectorLayer(r"https://github.com/echo-golf-oscar-romeo/kindergarten_accessibility/blob/c66a8368b58ad126afb051cc72958379104b80fb/living_houses_pts.geojson")
# kindergarten_pts = QgsVectorLayer(r"https://github.com/echo-golf-oscar-romeo/kindergarten_accessibility/blob/5f8e3b92d19082e424ded1010d9ca836e44ba49c/kindergarten_pts.geojson")

living_houses_pts = QgsVectorLayer(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\resid.geojson")
kindergarten_pts = QgsVectorLayer(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\kindergarten_pts.geojson")

In [6]:
from qgis.core import QgsProcessing
from qgis.core import QgsProcessingAlgorithm
from qgis.core import QgsProcessingMultiStepFeedback
from qgis.core import QgsProcessingParameterVectorLayer
from qgis.core import QgsProcessingParameterNumber
from qgis.core import QgsProcessingParameterFileDestination
from qgis.core import QgsProcessingParameterFeatureSink
import processing


class Kindergarten_egor(QgsProcessingAlgorithm):

    def initAlgorithm(self, config=None):
        self.addParameter(QgsProcessingParameterVectorLayer('livinghousescentoids', 'living_houses_centoids', types=[QgsProcessing.TypeVectorPoint], defaultValue=living_houses_pts))
        self.addParameter(QgsProcessingParameterVectorLayer('kindergartencentroids', 'kindergarten_centroids', types=[QgsProcessing.TypeVectorPoint], defaultValue=kindergarten_pts))
        self.addParameter(QgsProcessingParameterNumber('bufferradius', 'buffer_radius', type=QgsProcessingParameterNumber.Integer, minValue=1, defaultValue=300))
        self.addParameter(QgsProcessingParameterFileDestination('Spreadsheet', 'spreadsheet', fileFilter='Microsoft Excel (*.xlsx);;Open Document Spreadsheet (*.ods)', createByDefault=True, defaultValue=None))
        self.addParameter(QgsProcessingParameterFeatureSink('Intersection_raw', 'intersection_raw', optional=True, type=QgsProcessing.TypeVectorAnyGeometry, createByDefault=True, defaultValue=None))

    def processAlgorithm(self, parameters, context, model_feedback):
        # Use a multi-step feedback, so that individual child algorithm progress reports are adjusted for the
        # overall progress through the model
        feedback = QgsProcessingMultiStepFeedback(3, model_feedback)
        results = {}
        outputs = {}

        # Buffer
        alg_params = {
            'DISSOLVE': False,
            'DISTANCE': parameters['bufferradius'],
            'END_CAP_STYLE': 0,  # Round
            'INPUT': parameters['kindergartencentroids'],
            'JOIN_STYLE': 0,  # Round
            'MITER_LIMIT': 2,
            'SEGMENTS': 15,
            'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
        }
        outputs['Buffer'] = processing.run('native:buffer', alg_params, context=context, feedback=feedback, is_child_algorithm=True)

        feedback.setCurrentStep(1)
        if feedback.isCanceled():
            return {}

        # Join attributes by location
        alg_params = {
            'DISCARD_NONMATCHING': False,
            'INPUT': outputs['Buffer']['OUTPUT'],
            'JOIN': parameters['livinghousescentoids'],
            'JOIN_FIELDS': [''],
            'METHOD': 0,  # Create separate feature for each matching feature (one-to-many)
            'PREDICATE': [1],  # contains
            'PREFIX': 'j_',
            'OUTPUT': parameters['Intersection_raw']
        }
        outputs['JoinAttributesByLocation'] = processing.run('native:joinattributesbylocation', alg_params, context=context, feedback=feedback, is_child_algorithm=True)
        results['Intersection_raw'] = outputs['JoinAttributesByLocation']['OUTPUT']

        feedback.setCurrentStep(2)
        if feedback.isCanceled():
            return {}

        # Export to spreadsheet
        alg_params = {
            'FORMATTED_VALUES': False,
            'LAYERS': outputs['JoinAttributesByLocation']['OUTPUT'],
            'OVERWRITE': True,
            'USE_ALIAS': False,
            'OUTPUT': parameters['Spreadsheet']
        }
        outputs['ExportToSpreadsheet'] = processing.run('native:exporttospreadsheet', alg_params, context=context, feedback=feedback, is_child_algorithm=True)
        results['Spreadsheet'] = outputs['ExportToSpreadsheet']['OUTPUT']
        print(results['Spreadsheet'])
        return results

    def name(self):
        return 'kindergarten_egor'

    def displayName(self):
        return 'kindergarten_egor'

    def group(self):
        return 'egor'

    def groupId(self):
        return 'egor'

    def createInstance(self):
        return Kindergarten_egor()

In [14]:
base = Kindergarten_egor()
buffer_radius = 300
intersection_raw = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.geojson" 
table = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.xlsx"
parameters = {
    'bufferradius': buffer_radius,
    'kindergartencentroids': kindergarten_pts,
    'livinghousescentoids': living_houses_pts,
    'Intersection_raw': intersection_raw,
    'Spreadsheet': table
}
context = QgsProcessingContext()
feedback = QgsProcessingFeedback()
base.processAlgorithm(parameters, context=context, model_feedback=feedback)

brand_new_data = pd.read_excel(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.xlsx")
xb = xw.Book(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\Calculator_building_to_place.xlsx")
sheet = xb.sheets("intersection_raw")
sheet.range("A1", "K1000").value = brand_new_data
xb.save()
    
# do it again for one more house
    
base_1 = Kindergarten_egor()
df1 = pd.DataFrame(columns=['id', 'endowment'])
for i in range(52):    
    intersection_raw = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_000%s.geojson" % i
    table = r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_000%s.xlsx" % i
    parameters = {
        'bufferradius': buffer_radius,
        'kindergartencentroids': kindergarten_pts,
        'livinghousescentoids': QgsVectorLayer(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\id_%s.geojson" % i),
        'Intersection_raw': intersection_raw,
        'Spreadsheet': table
    }
    context = QgsProcessingContext()
    feedback = QgsProcessingFeedback()
    base_1.processAlgorithm(parameters, context=context, model_feedback=feedback)

    brand_new_data = pd.read_excel(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_000%s.xlsx" % i)
    xb = xw.Book(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\Calculator_building_to_place.xlsx")
    sheet = xb.sheets("intersection_raw(1)")
    sheet.clear()
    sheet.range("A1", "K1000").value = brand_new_data
    xb.save()
    
    endowment = round(pd.read_excel(r"C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\Calculator_building_to_place.xlsx", sheet_name = '10', engine="openpyxl")["Unnamed: 1"].loc[0], 2)
    temp = pd.DataFrame([[i, endowment]],columns=['id', 'endowment'])
    df1 =  pd.concat([df1, temp])
print(df1)

C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\trash\trash_0001.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0000.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0001.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0002.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0003.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0004.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0005.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0006.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0007.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\trash_0008.xlsx
C:\Users\egorb\Documents\ГП\2 курс\ПроектСем\3 module\PyModel\splitted\trash\tr

In [18]:
print(df1['endowment'])

0    42.94
0    42.94
0    42.94
0    42.79
0    42.99
0    42.81
0    43.07
0    42.83
0    43.03
0    42.72
0    42.89
0     42.6
0    42.69
0    42.94
0    42.67
0     43.0
0     43.0
0    42.69
0    42.94
0    42.94
0    43.01
0    43.21
0     42.9
0    42.94
0     43.0
0    42.96
0    42.94
0     42.9
0    42.88
0    42.93
0    42.95
0    42.88
0    42.94
0    42.82
0    42.99
0    43.21
0    42.82
0    42.94
0    42.84
0    42.95
0    42.85
0    42.97
0    42.95
0    42.99
0    42.95
0    42.94
0    42.95
0    42.96
0    42.91
0    42.83
0    42.93
0    42.88
Name: endowment, dtype: object
